In [1]:
import numpy as np
import pandas as pd
import mne
from scipy.stats import percentileofscore
import os

responses = ['n15','p30','n45','p60','mep']
pos_names = ['n15','p30','n45','p60','handknob']
usephase=False
rejcoil = False
models_path = fr"D:\REFTEP_ALL\Models_Aalto_Tuebingen_phase_{usephase}_rejcoil_{rejcoil}_depth0.8/"

def read_coefs_label(excelpath):
    datas = {}
    df = pd.read_csv(excelpath, header=None)
    for ind, _ in enumerate(df.iterrows()):
        if ind in [0,1]: #header row and intercept row
            continue
        row_now = df.iloc[ind].values.tolist()
        param_name = row_now[0]
        coef = np.float64(row_now[1])
        p = np.float64(row_now[6])
        datas[param_name] = {'coef':coef,'pval':p}
    return datas

def read_coefs_label_phase(excelpath):
    datas = {}
    df = pd.read_csv(excelpath, header=None)
    for ind, _ in enumerate(df.iterrows()):
        if ind in [0,1]: #header row and intercept row
            continue
        row_now = df.iloc[ind].values.tolist()
        param_name = row_now[0]
        if param_name == "phase_alpha":
            coef = np.float64(row_now[1])
            p = np.float64(row_now[6])
            datas[param_name] = {'coef':coef,'pval':p}
    return datas

In [2]:
usepsd=True
usecoil=True
usepac = False
usetime = [True, 'sample', {'Aalto':False,'Tuebingen':False}] #time scaling by ms according to mean iti
csv_ender = f'_usecoil_{usecoil}_usepsd_{usepsd}_usephase_{usephase}_usepac_{usepac}_usetime_{usetime[0]}_{usetime[1]}_grouptype_Subject_ref_site_None.csv'

In [3]:
gamma_params = []
for response in responses:
    print(response)
    datas_response = {}
    for parctype in ['aparc',str(['n15', 'p30', 'n45', 'p60', 'handknob'])]:
        if parctype == "aparc":
            labels = mne.read_labels_from_annot("fsaverage", parctype, "both", subjects_dir=r"D:\REFTEP_ALL\REFTEP_reco\Aalto_recon_all",verbose=False)
            labelnames_anat = [label.name for label in labels if "unknown" not in label.name]
            labelnames = labelnames_anat
        elif parctype == str(['n15', 'p30', 'n45', 'p60', 'handknob']):
            labelnames_func = [f'around_{pos_name}_label' for pos_name in pos_names if pos_name==response or pos_name=='handknob']
            labelnames = labelnames_func
        else:
            raise ValueError("bad labeltype")
        for name in labelnames:
            excelpath = f'{models_path}{response}_{name}{csv_ender}'
            datas = read_coefs_label(excelpath)
            datas_response[name] = datas
    param_names = list(datas.keys())
    for param in param_names:
        paramvals = []
        pvals = []
        for name in list(datas_response.keys()):
            paramvals.append(datas_response[name][param]['coef'])
            pvals.append(datas_response[name][param]['pval'])
            #param_locs.append(name)
        #inds = np.where(np.array(pvals) < 0.05)[0]
        inds = np.arange(len(pvals))
        if len(inds) > 0:
            paramvals = np.array(paramvals)[inds]
            for name_func in labelnames_func:
                percentile = percentileofscore(np.abs(paramvals),np.abs(datas_response[name_func][param]['coef']))
                if percentile >= 90:
                    print(response, param, name_func, percentile, datas_response[name_func][param]['coef'], len(inds))

    

    
        
        

n15
n15 PSD_gamma around_handknob_label 94.28571428571429 0.06080932629595374 70
n15 diff_coil around_n15_label 92.85714285714286 -0.0032361453827543605 70
n15 Latency around_n15_label 98.57142857142857 0.007904807194538201 70
n15 PSD_alpha_x_Latency around_handknob_label 91.42857142857143 0.007775193373954909 70
p30
p30 PSD_beta around_handknob_label 92.85714285714286 0.06469421439540265 70
p30 PSD_gamma around_p30_label 94.28571428571429 0.02331326696266481 70
p30 PSD_gamma around_handknob_label 97.14285714285714 0.02602596857319526 70
n45
n45 Subject Var around_n45_label 90.0 0.5816394180487409 70
p60
p60 PSD_gamma around_p60_label 95.71428571428572 0.036797736012607676 70
p60 PSD_gamma around_handknob_label 100.0 0.049742257523071653 70
mep
mep PSD_alpha around_handknob_label 97.10144927536231 0.11480054463617763 69
mep Subject Var around_handknob_label 100.0 0.7188142913865655 69


In [4]:
#analyze mu-phase and in sensorimotor region
parcnames_of_interest = ['around_handknob_label', 'precentral-lh','postcentral-lh']
gamma_params = []
for response in responses:
    #print(response)
    datas_response = {}
    for parctype in ['aparc',str(['n15', 'p30', 'n45', 'p60', 'handknob'])]:
        if parctype == "aparc":
            labels = mne.read_labels_from_annot("fsaverage", parctype, "both", subjects_dir=r"D:\REFTEP_ALL\REFTEP_reco\Aalto_recon_all",verbose=False)
            labelnames_anat = [label.name for label in labels if "unknown" not in label.name]
            labelnames = labelnames_anat
        elif parctype == str(['n15', 'p30', 'n45', 'p60', 'handknob']):
            labelnames_func = [f'around_{pos_name}_label' for pos_name in pos_names if pos_name==response or pos_name=='handknob']
            labelnames = labelnames_func
        else:
            raise ValueError("bad labeltype")
        labelnames = [labelname for labelname in labelnames if labelname in parcnames_of_interest]
        for name in labelnames:
            excelpath = f'{models_path}{response}_{name}{csv_ender}'
            datas = read_coefs_label_phase(excelpath)
            datas_response[name] = datas

    

    
        
        